### Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from zipfile import ZipFile

### Leitura dos arquivos

In [16]:
zip = ZipFile('PeNSE_2015_AMOSTRA2.zip')

In [17]:
zip.namelist()

['arquivos csv/',
 'arquivos csv/PENSE_AMOSTRA2_ALUNO.CSV',
 'arquivos csv/PENSE_AMOSTRA2_ALUNOESCOLA.CSV',
 'arquivos csv/PENSE_AMOSTRA2_ESCOLA.CSV',
 'arquivos sas/',
 'arquivos sas/pense_amostra2_aluno.sas7bdat',
 'arquivos sas/pense_amostra2_alunoescola.sas7bdat',
 'arquivos sas/pense_amostra2_escola.sas7bdat',
 'arquivos xlsx/',
 'arquivos xlsx/PENSE_AMOSTRA2_ALUNO.xlsx',
 'arquivos xlsx/PENSE_AMOSTRA2_ALUNOESCOLA.xlsx',
 'arquivos xlsx/PENSE_AMOSTRA2_ESCOLA.xlsx',
 'Dicionario_PENSE_Microdados_AMOSTRA2.xls']

In [18]:
caminho_zip = zip.namelist()[1]

In [19]:
dados = pd.read_csv(zip.open(caminho_zip), sep=';', thousands='.', decimal=',')
dados.head()

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,...,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,...,1223,1,299.735235,1,2,4,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,...,1223,1,355.170081,1,2,4,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,...,1223,1,299.735235,1,2,4,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,...,1223,1,355.170081,1,2,4,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,...,1223,1,299.735235,1,2,4,1,5,1,296


In [6]:
dados.columns

Index(['ANOPESQ', 'PAIS', 'REGEOGR', 'VB00004', 'VB01001', 'VB01002',
       'VB01003', 'VB01004', 'VB01005', 'VB01006',
       ...
       'ESTRATO_EXP', 'ESTRATOGEOREG', 'PESO', 'V0006', 'V0007', 'V0008',
       'V0041', 'aluno', 'escola', 'turma'],
      dtype='object', length=181)

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 181 entries, ANOPESQ to turma
dtypes: float64(10), int64(171)
memory usage: 22.9 MB


In [8]:
dados.shape

(16556, 181)

In [9]:
caminho_dicio = zip.namelist()[12]
dicio_dados = pd.read_excel(zip.open(caminho_dicio))
df_dict = dicio_dados.copy()
dicio_dados.head(20)

,VARIÁVEL,QUESTIONÁRIO DO ALUNO
0,ANOPESQ,Ano em que a pesquisa foi realizada
1,PAIS,País
2,76,Brasil
3,REGEOGR,Região geográfica
4,1,Norte
5,2,Nordeste
6,3,Sudeste
7,4,Sul
8,5,Centro-Oeste
9,VB00004,"Prezado(a) estudante, você concorda em partici..."


### Funções

**Vamos criar uma função que retornará um dicionário condicionando nossas perguntas e respostas**

In [10]:
# esta função retornará um dicionário que traduzirá os códigos de cada pergunta do questionário
# facilita para não ficarmos consultando as informações a todo momento
def dados_dict(pergunta, df=dicio_dados):
    try:
        result_dict = {}
        question = df[df['VARIÁVEL']==pergunta]['QUESTIONÁRIO DO ALUNO']
        result_dict['pergunta'] = question.values[0]

        lista = []
        indice = question.index[0] + 1
        alternativas = df.iloc[indice]['VARIÁVEL']

        while str(alternativas).isnumeric():
            lista.append([alternativas, df.iloc[indice]['QUESTIONÁRIO DO ALUNO']])
            indice += 1
            alternativas = df.iloc[indice]['VARIÁVEL']
            result_dict['Resp'] = dict(lista)

        return result_dict
    except IndexError:
        print('Erro de Indice. O código da questao está errado')

**Convertendo chaves de dicionário para tipo inteiro**

In [11]:
# esta função converterá as chaves dos dicionários de perguntas e respostas de string para inteiro
# só quando as chaves estiverem como inteiro que será possível mapear as informações
def dic_type_convert(dic):
    new_dic = {}
    for k, v in dic.items():
        if str(k).isnumeric():
            k = int(k)
            new_dic[k] = v
        
    return new_dic

**Função para criação de tabela de frequência**

In [52]:
def tab_freq(cod): 
    indice = dados[cod].value_counts().index.map(dic_type_convert(dados_dict(cod)['Resp']))
    valores = dados[cod].value_counts().values
    percent = round(dados[cod].value_counts(normalize=True)*100, 2).values
    df = pd.DataFrame({'frequencia':valores, 'percent (%)':percent}, index=indice)
    return df

### Análise Exploratória

Vamos verificar quantas perguntas existem no questionário e quantas pessoas responderam

In [64]:
print(f'{dados.shape[0]} alunos participaram da pesquisa \n{dados.shape[1]} perguntas foram respondidas')

16556 alunos participaram da pesquisa 
181 perguntas foram respondidas


In [78]:
dicio_dados[dicio_dados['VARIÁVEL'].str.contains('VB01')==True]['QUESTIONÁRIO DO ALUNO'].to_list()

['Qual é o seu sexo?',
 'Qual é a sua cor ou raça?',
 'Qual é a sua idade?',
 'Qual é o mês do seu aniversário?',
 'Em que ano você nasceu?',
 'Em que ano/série você está?',
 'Em que turno você estuda?',
 'Você estuda em regime integral (tem atividades escolares por 7 horas ou mais horas diárias, durante todo o período escolar)?',
 'Você estuda em regime de internato (a escola possui alojamento onde os alunos permanecem e dormem diariamente, durante todo o período escolar)?',
 'Qual o grau de escolaridade mais elevado que você pretendo concluir?',
 'Quando terminar o ciclo/curso que você está frequentando atualmente, você pretende?',
 'Você mora com sua mãe?',
 'Você mora com seu pai?',
 'Contando com você, quantas pessoas moram na sua casa ou apartamento?',
 'Na sua casa tem telefone fixo (convencional)?',
 'Você tem celular?',
 'Na sua casa tem computador (de mesa, ou netbook, laptop, etc)?',
 'Você tem acesso à internet em sua casa?',
 'Alguém que mora na sua casa tem carro?',
 'Alg

**Perguntas selecionadas**

In [ ]:
lista_perguntas = ['VB01001', 'VB01002', 'VB01003', 'VB01021', 'VB01022', 'VB01025', 'VB01026', ]

In [143]:
# fazer uma coluna para saber quem mora com somente um dos pais
mae = dados['VB01006'].map(dic_type_convert(dados_dict('VB01006')['Resp']))
pai = dados['VB01007'].map(dic_type_convert(dados_dict('VB01007')['Resp']))
pais = pd.DataFrame({'mae':mae, 'pai':pai})
pais['2_pais'] = pais['mae'] == pais['pai']
pais['2_pais'] = pais['2_pais'].map({False:'Não', True:'Sim'})
pais.sample(10)

,mae,pai,2_pais
3967,Sim,Não,Não
4181,Não,Não,Não
9480,Sim,Sim,Sim
2326,Sim,Sim,Sim
2874,Sim,Sim,Sim
8886,Sim,Sim,Sim
2075,Sim,Não,Não
13132,Sim,Sim,Sim
1613,Sim,Sim,Sim
6715,Sim,Sim,Sim


In [200]:
idx = pd.Index(dados['VB01002'].map(dic_type_convert(dados_dict('VB01002')['Resp'])), name='index')
c = round(pd.crosstab(index=idx, columns=pais['2_pais'], normalize='index')*100, 2)
c = c.rename_axis('cor_raça', axis='index')
c = c.rename_axis('pais_juntos', axis='columns')

In [201]:
c

pais_juntos,Não,Sim
cor_raça,,
Amarela,41.01,58.99
Branca,37.48,62.52
Indígena,41.31,58.69
Não informado,34.78,65.22
Parda,43.87,56.13
Preta,46.67,53.33


In [205]:
idx = pd.Index(dados['VB01008A'].map(dic_type_convert(dados_dict('VB01008A')['Resp'])), name='index')
c = round(pd.crosstab(index=idx, columns=pais['2_pais'], normalize='index')*100, 2)
c = c.rename_axis('pretensao_mae', axis='index')
c = c.rename_axis('pais_juntos', axis='columns')
c

pais_juntos,Não,Sim
pretensao_mae,,
"Minha mãe começou a faculdade (ensino superior), mas não terminou",45.64,54.36
"Minha mãe começou o ensino fundamental ou 1º grau, mas não terminou",42.41,57.59
"Minha mãe começou o ensino médio ou 2º grau, mas não terminou",44.11,55.89
Minha mãe não estudou,46.86,53.14
Minha mãe terminou a faculdade (ensino superior),35.04,64.96
Minha mãe terminou o ensino fundamental ou 1º grau,41.68,58.32
Minha mãe terminou o ensino médio ou 2° grau,40.56,59.44
Não informado,35.29,64.71
Não sei,43.57,56.43
